In [ ]:
model_name = "slall/opt-125M-imdb-lora-adapter-merged"

In [ ]:
from openai import OpenAI
# Modify OpenAI's API key and API base to use vLLM's API server.
openai_api_key = "EMPTY"
openai_api_base = "http://localhost:8000/v1"
client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)
completion = client.completions.create(model=model_name,
                                      prompt="San Francisco is a")
print("Completion result:", completion)

In [ ]:
pipe = pipeline("feature-extraction")
#pipe.save_pretrained("distilroberta-base")

In [ ]:
from transformers import pipeline

feature_extraction = pipeline('feature-extraction', model="/home/azureuser/distilroberta-base", tokenizer="/home/azureuser/distilroberta-base")
embeddings = feature_extraction(["Hello I'm a single sentence",
                               "And another sentence",
                               "And the very very last one"])
embeddings

In [ ]:
from vllm import LLM, SamplingParams
llm = LLM(model=model_name)

In [ ]:
from langchain_community.llms import VLLM
llm = VLLM(
    model=model_name,
    trust_remote_code=True,  # mandatory for hf models
    max_new_tokens=128,
    top_k=10,
    top_p=0.95,
    temperature=0.8,
)

print(llm.invoke("What is the capital of France ?"))

In [ ]:
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate

template = """Question: {question}

Answer: Let's think step by step."""
prompt = PromptTemplate.from_template(template)

llm_chain = LLMChain(prompt=prompt, llm=llm)

question = "Who was the US president in the year the first Pokemon game was released?"

print(llm_chain.invoke(question))

In [ ]:
# LORA NOT SUPPORTED ON CPU
from langchain_community.llms import VLLM
from vllm.lora.request import LoRARequest

llm = VLLM(
    model="facebook/opt-125m",
    top_k=1,
    top_p=0.90,
    temperature=0.1,
    vllm_kwargs={
        "enable_lora": True,
        "max_model_len": 350,
    },
)

LoRA_ADAPTER_PATH = "path/to/adapter"
lora_adapter = LoRARequest("lora_adapter", 1, LoRA_ADAPTER_PATH)

print(
    llm.invoke("What are some popular Korean street foods?", lora_request=lora_adapter)
)

## Vector DB / Embeddings example

In [ ]:
!pip install PyMuPDF langchain-community 

In [ ]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

In [ ]:
from langchain_community.document_loaders import PyMuPDFLoader

file_path = "nke-10k-2023.pdf"
loader = PyMuPDFLoader(file_path)

docs = loader.load()

print(len(docs))

In [ ]:
print(f"{docs[0].page_content[:200]}\n")
print(docs[0].metadata)

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

len(all_splits)

In [ ]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(feature_extraction)

In [ ]:
ids = vector_store.add_documents(documents=all_splits)

In [17]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="distilbert/distilroberta-base") #model="/home/azureuser/distilroberta-base"

text = "This is a test document."
query_result = embeddings.embed_query(text)

No sentence-transformers model found with name distilbert/distilroberta-base. Creating a new one with mean pooling.


In [ ]:
query_result